In [4]:
API_KEY = 'y5Pt4OyhvXKRMNgmQUudrQTVp'
ACESS_TOKEN = '1536862091537178624-BRUEJHXCKJkb9EQP4eMunnmAyP6TDT'
API_KEY_SECRET = 'H2bRlfYoQ9appGSqiQvSONvyitEuttt87fPtYlYhlmYtsRzR5A'
ACESS_TOKEN_SECRET = 'Q6vzz1YsRxOhJ1mDHOXVSt1Id687GuwL1l1yhkRm92SQi'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAI4ZdwEAAAAAr2Z9ihJUyjN2Z5aAsiVFH0zt%2FIM%3DtUFLGD7Fo30mYT5hj2IJwVex27bCSKz2ZAgQDssNKs6HssNBD6'

In [5]:
import sys
!{sys.executable} -m pip install pandas

You should consider upgrading via the 'C:\Users\eduar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
import sys
!{sys.executable} -m pip install tweepy

You should consider upgrading via the 'C:\Users\eduar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import tweepy
import pandas as pd
import os  

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [8]:
def pesquisa_politico(politico):
    data = client.search_recent_tweets(query=politico +' -is:retweet',max_results=100,tweet_fields=['created_at','lang','public_metrics','source','geo'],place_fields=['country','geo','name','country_code'],expansions=['geo.place_id','author_id'])
    return data

In [9]:
def preenche_listas(data,text,id,retweets,replys,likes,author_id,source,lang,created_at,place_id,geo_full_name,geo_x1,geo_x2,geo_y1,geo_y2,geo_id):
    for e in data:
        if isinstance(e,dict):
            for i in e.keys():
                if i == 'places':
                    for l in e[i]:
                        geo_full_name.append(l.full_name)
                        geo_x1.append(l.data['geo']['bbox'][1])
                        geo_x2.append(l.data['geo']['bbox'][0])
                        geo_y1.append(l.data['geo']['bbox'][3])
                        geo_y2.append(l.data['geo']['bbox'][2])
                        geo_id.append(l.id)
        if isinstance(e,list):
            for i in e:
                place_id.append(i.geo)
                created_at.append(i.created_at)
                lang.append(i.lang)
                source.append(i.source)
                author_id.append(i.author_id)
                retweets.append(i.public_metrics['retweet_count'])
                likes.append(i.public_metrics['like_count'])
                replys.append(i.public_metrics['reply_count'])
                id.append(i.id)
                text.append(i.text)

In [10]:
def salvar_100(politico, df_tweet, df_tweet_geo):
    os.makedirs('dados/ultimos 100 tweets', exist_ok=True)  
    df_tweet.to_csv('dados/ultimos 100 tweets/'+politico+'_tweets.csv') 
    df_tweet_geo.to_csv('dados/ultimos 100 tweets/'+politico+'_tweets_locais.csv') 

In [11]:
def criar_csv_100tweets(politico):
    data = pesquisa_politico(politico)
    text = []
    id = []
    retweets = []
    replys = []
    likes = []
    author_id = []
    source = []
    lang = []
    created_at = []
    place_id = []
    geo_full_name = []
    geo_x1 = []
    geo_x2 = []
    geo_y1 = []
    geo_y2 = []
    geo_id = []
    preenche_listas(data,text,id,retweets,replys,likes,author_id,source,lang,created_at,place_id,geo_full_name,geo_x1,geo_x2,geo_y1,geo_y2,geo_id)
    df_tweet = pd.DataFrame(data={'text' : text, 'id' : id, 'retweets' : retweets, 'replys' : replys,'likes' : likes, 'author_id' : author_id, 'source' : source, 'lang' : lang, 'created_at': created_at,'place_id' : place_id})
    df_tweet_geo = pd.DataFrame(data={'geo_full_name' : geo_full_name,'geo_id' : geo_id, 'geo_x1' : geo_x1, 'geo_x2' : geo_x2, 'geo_y1' : geo_y1, 'geo_y2' : geo_y2})
    salvar_100(politico, df_tweet,df_tweet_geo)

In [12]:
pre_candidatos = ['bolsonaro', 
                    'lula', 
                    'ciro', 
                    'simone tebet', 
                    'bivar', 
                    'Luiz Felipe d’Avila', 
                    'janones', 
                    'leonardo pericles', 
                    'pablo maçal', 
                    'vera lucia', 
                    'sofia manzano', 
                    'eymael']

In [20]:
for c in pre_candidatos:
    criar_csv_100tweets(c)

In [13]:
def pesquisa_p_t_count(politico,tema):
    counts = client.get_recent_tweets_count(query=politico+' '+tema+' -is:retweet',granularity='day')
    return counts.data

In [19]:
def salvar_7(politico, df_tweet, tipo):
    os.makedirs('dados/ultimos 7 dias', exist_ok=True)  
    if len(tipo) == 0:
        df_tweet.to_csv('dados/ultimos_7_dias/'+politico+'_por_tema_tweets.csv')
    else:
        df_tweet.to_csv('dados/ultimos_7_dias/'+tipo+'/'+politico+'_por_tema_tweets.csv') 

In [15]:
def criar_csv_7dias(politico, temas, tipo = ''):
    valor_tema_dia = []
    for i in range(len(temas)):
        valor_tema_dia.append([])
        pesquisa = pesquisa_p_t_count(politico,temas[i])
        if i == 0:
            dias = []
        for j in range(len(pesquisa)):
            valor_tema_dia[i].append(pesquisa[j]['tweet_count'])
            if i == 0:
                dias.append(pesquisa[j]['start'])
    datas = dict(zip(temas, valor_tema_dia))
    df_tweet = pd.DataFrame(data=datas,index=dias)
    print(df_tweet)
    salvar_7(politico, df_tweet, tipo)

In [43]:
temas = ['ditadura',
        'reforma trabalhista',
        'feminismo',
        'reforma agrária',
        'corrupção',
        'segurança']

In [44]:
for pre_candidato in pre_candidatos:
    criar_csv_7dias(pre_candidato, temas)

                          ditadura  reforma trabalhista  feminismo  \
2022-06-10T21:40:28.000Z        14                    6          0   
2022-06-11T00:00:00.000Z       152                   12          5   
2022-06-12T00:00:00.000Z       116                   13          3   
2022-06-13T00:00:00.000Z       175                    4          6   
2022-06-14T00:00:00.000Z       132                   19          3   
2022-06-15T00:00:00.000Z       168                   19          2   
2022-06-16T00:00:00.000Z       126                    6          7   
2022-06-17T00:00:00.000Z       104                    4          7   

                          reforma agrária  corrupção  segurança  
2022-06-10T21:40:28.000Z                2         55         50  
2022-06-11T00:00:00.000Z                5        374        134  
2022-06-12T00:00:00.000Z               12        460        194  
2022-06-13T00:00:00.000Z                9        571        237  
2022-06-14T00:00:00.000Z               

In [20]:
negativos = ['(corrupto OR corrupta)',
            'machista',
            'racista',
            'nazista',
            'fascista',
            '(burro OR burra)',
            'comunista',
            'socialista',
            'anarquista',
            '(alienado OR alienada)',
            '(analfabeto OR analfabeta)',
            '(mentiroso OR mentirosa)',
            '(desonesto OR desonesta)',
            '(safado OR safada)',
            'canalha',
            '(ladrão OR ladra)',
            'hipócrita',
            'genocida',
            '(ditador OR ditadora)',
            '(autoritário OR autoritária)',
            'sexista',
            'populista',
            '(maconheiro OR maconheira)',
            'imbecil',
            '(vagabundo OR vagabunda)']

In [21]:
for pre_candidato in pre_candidatos:
    criar_csv_7dias(pre_candidato, negativos, tipo='neg')

                          (corrupto OR corrupta)  machista  racista  nazista  \
2022-06-14T01:16:49.000Z                     286         7       23       35   
2022-06-15T00:00:00.000Z                     312        19       24       26   
2022-06-16T00:00:00.000Z                     355        19       31      102   
2022-06-17T00:00:00.000Z                     408        21       23       46   
2022-06-18T00:00:00.000Z                     342        10       39       26   
2022-06-19T00:00:00.000Z                     404        10       31       37   
2022-06-20T00:00:00.000Z                     412        21       31       52   
2022-06-21T00:00:00.000Z                      52         1        1        1   

                          fascista  (burro OR burra)  comunista  socialista  \
2022-06-14T01:16:49.000Z       111               295        128          35   
2022-06-15T00:00:00.000Z       125               240        118          35   
2022-06-16T00:00:00.000Z       130        

                          (corrupto OR corrupta)  machista  racista  nazista  \
2022-06-14T01:17:04.000Z                      54         3        6        3   
2022-06-15T00:00:00.000Z                      55         1        0        5   
2022-06-16T00:00:00.000Z                      69         2       13        2   
2022-06-17T00:00:00.000Z                      84         4        6        7   
2022-06-18T00:00:00.000Z                      43         1        1        1   
2022-06-19T00:00:00.000Z                      56         4        2        2   
2022-06-20T00:00:00.000Z                      55         2        1        8   
2022-06-21T00:00:00.000Z                       2         0        0        0   

                          fascista  (burro OR burra)  comunista  socialista  \
2022-06-14T01:17:04.000Z        47                36          9           5   
2022-06-15T00:00:00.000Z        39                32         14           1   
2022-06-16T00:00:00.000Z        24        

                          (corrupto OR corrupta)  machista  racista  nazista  \
2022-06-14T01:17:18.000Z                      18         0        0        0   
2022-06-15T00:00:00.000Z                       6         0        0        0   
2022-06-16T00:00:00.000Z                       2         0        0        0   
2022-06-17T00:00:00.000Z                       3         0        0        0   
2022-06-18T00:00:00.000Z                       2         0        0        0   
2022-06-19T00:00:00.000Z                       1         0        0        0   
2022-06-20T00:00:00.000Z                       1         0        0        0   
2022-06-21T00:00:00.000Z                       0         0        0        0   

                          fascista  (burro OR burra)  comunista  socialista  \
2022-06-14T01:17:18.000Z        60                 1          1           3   
2022-06-15T00:00:00.000Z        13                 2          0           1   
2022-06-16T00:00:00.000Z         3        

                          (corrupto OR corrupta)  machista  racista  nazista  \
2022-06-14T01:17:32.000Z                       0         0        0        0   
2022-06-15T00:00:00.000Z                       0         0        0        0   
2022-06-16T00:00:00.000Z                       0         0        0        0   
2022-06-17T00:00:00.000Z                       0         0        0        0   
2022-06-18T00:00:00.000Z                       0         0        0        0   
2022-06-19T00:00:00.000Z                       0         0        0        0   
2022-06-20T00:00:00.000Z                       0         0        0        0   
2022-06-21T00:00:00.000Z                       0         0        0        0   

                          fascista  (burro OR burra)  comunista  socialista  \
2022-06-14T01:17:32.000Z         0                 0          0           0   
2022-06-15T00:00:00.000Z         1                 0          1           0   
2022-06-16T00:00:00.000Z         0        

                          (corrupto OR corrupta)  machista  racista  nazista  \
2022-06-14T01:17:47.000Z                       0         0        0        0   
2022-06-15T00:00:00.000Z                       0         0        0        0   
2022-06-16T00:00:00.000Z                       0         0        0        0   
2022-06-17T00:00:00.000Z                       0         0        0        0   
2022-06-18T00:00:00.000Z                       0         0        0        0   
2022-06-19T00:00:00.000Z                       0         0        0        0   
2022-06-20T00:00:00.000Z                       0         0        0        0   
2022-06-21T00:00:00.000Z                       0         0        0        0   

                          fascista  (burro OR burra)  comunista  socialista  \
2022-06-14T01:17:47.000Z         0                 0          0           0   
2022-06-15T00:00:00.000Z         0                 0          0           0   
2022-06-16T00:00:00.000Z         0        

                          (corrupto OR corrupta)  machista  racista  nazista  \
2022-06-14T01:18:01.000Z                       0         0        0        0   
2022-06-15T00:00:00.000Z                       0         0        0        0   
2022-06-16T00:00:00.000Z                       0         0        0        0   
2022-06-17T00:00:00.000Z                       0         0        0        0   
2022-06-18T00:00:00.000Z                       0         0        0        0   
2022-06-19T00:00:00.000Z                       0         0        0        0   
2022-06-20T00:00:00.000Z                       0         0        0        0   
2022-06-21T00:00:00.000Z                       0         0        0        0   

                          fascista  (burro OR burra)  comunista  socialista  \
2022-06-14T01:18:01.000Z         0                 0          0           0   
2022-06-15T00:00:00.000Z         0                 0          0           0   
2022-06-16T00:00:00.000Z         0        

In [48]:
positivos = ['(honesto OR honesta)',
            '(democrático OR democrática)',
            'inteligente',
            'consciente',
            '(culto OR culta)',
            '(correta OR correto)']

In [51]:
for pre_candidato in pre_candidatos:
    criar_csv_7dias(pre_candidato, positivos, tipo='pos')

                          (honesto OR honesta)  (democrático OR democrática)  \
2022-06-10T22:42:30.000Z                    36                            13   
2022-06-11T00:00:00.000Z                   167                           100   
2022-06-12T00:00:00.000Z                   200                            94   
2022-06-13T00:00:00.000Z                   169                            82   
2022-06-14T00:00:00.000Z                   163                            67   
2022-06-15T00:00:00.000Z                   119                            45   
2022-06-16T00:00:00.000Z                   141                            66   
2022-06-17T00:00:00.000Z                   140                            43   

                          inteligente  consciente  (culto OR culta)  \
2022-06-10T22:42:30.000Z            7           0                 1   
2022-06-11T00:00:00.000Z           66           7                14   
2022-06-12T00:00:00.000Z           56           7                1